Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2020-05-12 23:43:28--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.8.181
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.8.181|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  17.1MB/s    in 13s     

2020-05-12 23:43:42 (15.0 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [0]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [0]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [0]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

# 1. What is the end goal:
- Given the 10 most frequently ordered products we are trying to find exactly how many times each of these products were ordered.


# 2. Find which coloumns out of all the datasets will be the most useful in our search.

- Import the libraries will will need

In [0]:
# Can't forget the imports!!!!
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


- Examine all of our datasets to find the best suited columns.
    - To do so I created a function to check for every useful csv file in the directory and hide all the dot files.
    - Then display the head of all the data frames stored in the csv files.

In [0]:
# Listing all the files in the current working directory:
%cd instacart_2017_05_01/
! ls

/content/instacart_2017_05_01
aisles.csv	 order_products__prior.csv  orders.csv
departments.csv  order_products__train.csv  products.csv


In [0]:
# Create a variable for the files path.
file_dir = r'/content/instacart_2017_05_01'

# Create a function to list files from a directory and keep the '.'files hidden,
# and display them as a pandas DataFrame.
def listFiles(file_dir):
    hidden_files = []
    cleaned_files = []
    for files in os.listdir(file_dir):
        if files.startswith('.'):
            hidden_files.append(files)
        else:
            cleaned_files.append(files)                
    [display(pd.read_csv(i).head(3)) for i in cleaned_files]


# Use the new function to list the files in desired directory.
# listFiles(file_dir)

- After looking at the data, we can rule out the tables that won't aid us in solving the problem at hand:
    - **Exclude List**:
      1. `aisle.csv`
      2. `departments.csv`
      
    - **Include List**:  
      1. `order_products__prior.csv`
      2. `order_products__train.csv`
      3. `orders.csv`
      4. `products`

In [0]:
# Read in our 4 csv files from the 'Include List':
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')
opp = pd.read_csv('order_products__prior.csv')
opt = pd.read_csv('order_products__train.csv')

In [0]:
# display 3 samples of orders
products.head(3)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7


In [0]:
# display 3 samples of orders
orders.sample(3)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
595249,3146264,35879,prior,15,4,9,6.0
1956426,2739895,117460,prior,66,4,17,3.0
1026600,2582518,61716,prior,4,0,23,30.0


In [0]:
# dislay top 3 of opp
opp.head(3)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0


In [0]:
# display top 3 of opt
opt.head(20)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
5,1,13176,6,0
6,1,47209,7,0
7,1,22035,8,1
8,36,39612,1,0
9,36,19660,2,1


- Now that we have our desired tables, let us take a look at which columns will be of aid to us.
    - Data frame & Columns of use:
      1. **products**
        - `product_id`
        - `product_name`
      2. **orders**
        - `order_number`
        - `order_id`
      3. **opp**
        - `order_id`
        - `product_id`
      4. **opt**
        - `order_id`
        - `product_id`    

- Next lets create smaller data frames useing our desired columns.

In [0]:
# Create new data frames by pulling out the desired variables
opt = opt[['order_id','product_id']]
opp = opp[['order_id','product_id']]
orders = orders[['order_id','order_number']]
products = products[['product_id','product_name']]

In [0]:
# display opt:
display(opt.head(3))
# display opp:
display(opp.head(3))
# display orders:
display(orders.head(3))
# display products:
display(products.head(3))

,order_id,product_id
0,1,49302
1,1,11109
2,1,10246


,order_id,product_id
0,2,33120
1,2,28985
2,2,9327


,order_id,order_number
0,2539329,1
1,2398795,2
2,473747,3


,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea


# 3. Merge our data into one table.

In [0]:
# merge opp and opt togeter this will act as the key for our other data frames
opp_opt = pd.merge_ordered(opp,opt)

In [0]:
# merge opp_opt and products
glutten_table = pd.merge(opp_opt,products[['product_id','product_name']],
                         on='product_id',
                         how='inner')

In [0]:
# last merge glutten_table and orders 
last_glutten_table = pd.merge(glutten_table,orders[['order_id','order_number']],
                              on='order_id',
                              how='inner')

In [0]:
# we now have a dataframe that will give us the information to see the 
# most frequent purchased items.
last_glutten_table.head()

,order_id,product_id,product_name,order_number
0,1,10246,Organic Celery Hearts,4
1,1,11109,Organic 4% Milk Fat Whole Milk Cottage Cheese,4
2,1,13176,Bag of Organic Bananas,4
3,1,22035,Organic Whole String Cheese,4
4,1,43633,Lightly Smoked Sardines in Olive Oil,4


# 4. Lets now get our top ordered products.

Top 10 ordered product should be:
1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

In [0]:
# Now by counting the values in our 'product_name' column.
last_glutten_table['product_name'].value_counts().head(10)

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64



---



---





---



---



## Reshape Data Section

- Replicate the lesson code. Remember, if you haven't followed along typing out what we did during lecture, do that now to get more repetition with the syntax.
- Take table 2 (the transpose of table 1) and practice changing it into Tidy format and back again. You should not use the transpose operation anywhere in this code:
 - Table 2 --> Tidy
 - Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

**1. Make Tidy Table from Wide Table**


In [0]:
import pandas as pd
import numpy as np

table_1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table_2 = table_1
table_2

,treatmenta,treatmentb
John Smith,NaN,2
Jane Doe,16.0,11
Mary Johnson,3.0,1


- We can use the pandas melt function to reshape Table 2 into Tidy format.

In [0]:
# Get the column names as a list.
cols = table_2.columns.to_list()
# Now get the index values as another list.
rows = table_2.index.to_list()

In [0]:
# For table_2, conver the index into a column using the 'reset_index' method.
table_2 = table_2.reset_index()
table_2

,index,treatmenta,treatmentb
0,John Smith,NaN,2
1,Jane Doe,16.0,11
2,Mary Johnson,3.0,1


In [0]:
# Convert the table from 'wide' to 'tidy' format using th 'melt' method.
tidy_2 = table_2.melt(id_vars='index',value_vars=['treatmenta','treatmentb'])
tidy_2

,index,variable,value
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [0]:
# Rename the columns
tidy_2 = tidy_2.rename(columns={
    'index':'name',
    'variable':'trt',
    'value':'result'
})
tidy_2

,name,trt,result
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [0]:
# Shorten the 'trt' values
tidy_2['trt'] = tidy_2['trt'].str.replace('treatment','')
tidy_2

,name,trt,result
0,John Smith,a,NaN
1,Jane Doe,a,16.0
2,Mary Johnson,a,3.0
3,John Smith,b,2.0
4,Jane Doe,b,11.0
5,Mary Johnson,b,1.0


**2. Make Wide Table from Tidy Table**

In [0]:
# all we need to do is pivot.
wide_2 = tidy_2.pivot_table(index='name',
                            columns='trt',
                            values='result')
wide_2

trt,a,b
name,,
Jane Doe,16.0,11.0
John Smith,NaN,2.0
Mary Johnson,3.0,1.0


**4. Load seaborn's flights dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.**


In [0]:
# load in flights dataset
flights = sns.load_dataset('flights')

In [0]:
# display our flights table
wide_flights = flights.pivot_table(index='year',
                                   columns='month',
                                   values='passengers')
wide_flights

month,January,February,March,April,May,June,July,August,September,October,November,December
year,,,,,,,,,,,,
1949,112,118,132,129,121,135,148,148,136,119,104,118
1950,115,126,141,135,125,149,170,170,158,133,114,140
1951,145,150,178,163,172,178,199,199,184,162,146,166
1952,171,180,193,181,183,218,230,242,209,191,172,194
1953,196,196,236,235,229,243,264,272,237,211,180,201
1954,204,188,235,227,234,264,302,293,259,229,203,229
1955,242,233,267,269,270,315,364,347,312,274,237,278
1956,284,277,317,313,318,374,413,405,355,306,271,306
1957,315,301,356,348,355,422,465,467,404,347,305,336


In [0]:
# i tried a bunch of different conditions ot see if i could get it to confirm 
# some numbers foe me haha but yet to no avail.
condition = (wide_flights.index.values==1949)
wide_flights[condition]

month,January,February,March,April,May,June,July,August,September,October,November,December
year,,,,,,,,,,,,
1949,112,118,132,129,121,135,148,148,136,119,104,118


In [0]:
condition

array([ True, False, False, False, False, False, False, False, False,
       False, False, False])

## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



- **First**
  - Load in the instacart data.

In [0]:
# check current directory 
%pwd

'/content'

In [0]:
# list all the files in our directory 
%cd instacart_2017_05_01/
%ls

/content/instacart_2017_05_01
aisles.csv       order_products__prior.csv  orders.csv
departments.csv  order_products__train.csv  products.csv


In [0]:
# Load all the files as pandas dataframes using the listFiles function i made earlier.
file_dir = r'/content/instacart_2017_05_01'
listFiles(file_dir)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0


,department_id,department
0,1,frozen
1,2,other
2,3,bakery


- To get the most recent orders first we need to get the columns of data that will be useful in our search.

In [0]:
# make a new dataframe for orders:
orders = pd.read_csv('orders.csv')
orders = orders[['order_id','order_number','order_dow','order_hour_of_day']]
# check out new dataframe:
orders.head()

,order_id,order_number,order_dow,order_hour_of_day
0,2539329,1,2,8
1,2398795,2,3,7
2,473747,3,3,12
3,2254736,4,4,7
4,431534,5,4,15


In [0]:
# make a new dataframe for products:
products_info = pd.read_csv('products.csv') 
products_info = products_info[['product_id','product_name']]
# check new data frame:
products_info.head()

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [0]:
# make a new dataframe for train data:
orders_train = pd.read_csv('order_products__train.csv')
# make a new dataframe for prior data:
orders_prior = pd.read_csv('order_products__prior.csv')
# concatenate the these 2 data frames to get all of the reordered data and its
# product_id and order_id:
otp = pd.concat([orders_train,orders_prior], axis=0, ignore_index=True)
# dropping the add_to_cart_order since its not needed, because we are looking 
# for placed orders:
otp.drop('add_to_cart_order',axis=1,inplace=True)

In [0]:
# check if change took place:
otp.head()

,order_id,product_id,reordered
0,1,49302,1
1,1,11109,1
2,1,10246,0
3,1,49683,0
4,1,43633,1


In [0]:
# now lets merge otp and products.
otp_products = pd.merge(otp,products_info,
                         on='product_id',
                         how='inner')
# check out our new data frame
otp_products.head()

,order_id,product_id,reordered,product_name
0,1,49302,1,Bulgarian Yogurt
1,816049,49302,1,Bulgarian Yogurt
2,1242203,49302,1,Bulgarian Yogurt
3,1383349,49302,1,Bulgarian Yogurt
4,1787378,49302,0,Bulgarian Yogurt


In [0]:
# now for the final merge:
most_recent_orders = pd.merge(orders[['order_id','order_number']],otp_products[['order_id','reordered','product_name']],
                              how='inner',on='order_id')
# check out some samples from our new dataframe
most_recent_orders.sample(15)

TypeError: ignored

1    19955360
0    13863746
Name: reordered, dtype: int64

In [0]:
aisles = pd.read_csv('aisles.csv')
aisles.head(3)

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars


In [0]:
cross_tab = pd.crosstab(most_recent_orders['reordered'],most_recent_orders['product_name'],margins=True,normalize=True)

In [0]:
sns.relplot(x='product_name',
            y='reordered',
            hue='order_number',
            data=most_recent_orders);

/usr/local/lib/python3.6/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 143 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/usr/local/lib/python3.6/dist-packages/seaborn/axisgrid.py:856: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  self.fig.tight_layout()


KeyboardInterrupt: ignored

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####